In [1]:
from MaxAgents import MaxRandomAgent
from MinAgents import MinRandomAgent
from Model import MyModel, CustomDataCollector
from math import floor

## Parameters 
N = 10
prop_adversaries = .2
coin_toss = .9
rounds_factor = 10

model = MyModel(N, coin_toss, prop_adversaries, MaxRandomAgent, MinRandomAgent)

for i in range(floor(N * rounds_factor)):
    model.step()

df = CustomDataCollector(model)

In [4]:
df.head(30)

,Pairs,Voting,Outcome,Action
0,"(0, 9)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
1,"(1, 3)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[1, 0]"
2,"(2, 3)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[0, 0]"
3,"(3, 4)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[0, 1]"
4,"(4, 6)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
5,"(5, 7)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
6,"(6, 9)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
7,"(7, 6)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
8,"(8, 3)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[1, 0]"
9,"(9, 4)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
